In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix


def build_user_game_matrix(df, min_games=5, min_users=5):
    game_counts = df['item_name'].value_counts()
    popular_games = game_counts[game_counts >= min_games].index
    df = df[df['item_name'].isin(popular_games)]
    user_counts = df['user_id'].value_counts()
    active_users = user_counts[user_counts >= min_users].index
    df = df[df['user_id'].isin(active_users)]
    pivot_table = df.pivot_table(index='user_id', columns='item_name', values='playtime_forever', fill_value=0)
    sparse_matrix = csr_matrix(pivot_table.values)
    return sparse_matrix, pivot_table


def collaborative_filtering_recommendations(user_id, user_game_matrix, pivot_table, n_recommendations=5):
    try:
        user_index = list(pivot_table.index).index(user_id)
    except ValueError:
        print(f"用户 {user_id} 不在数据集中。")
        return []
    similarity = cosine_similarity(user_game_matrix[user_index], user_game_matrix).flatten()
    similar_users = similarity.argsort()[::-1][1:n_recommendations + 1]
    similar_users_ids = pivot_table.index[similar_users]
    recommendations = set()
    for similar_user in similar_users_ids:
        recommendations.update(pivot_table.loc[similar_user][pivot_table.loc[similar_user] > 0].index)
    played_games = set(pivot_table.loc[user_id][pivot_table.loc[user_id] > 0].index)
    recommendations = recommendations - played_games 
    return list(recommendations)[:n_recommendations]


def main_task_two(df, user_id):

    user_game_matrix, pivot_table = build_user_game_matrix(df)
    recommendations = collaborative_filtering_recommendations(user_id, user_game_matrix, pivot_table)
    print("协同过滤推荐：", recommendations)

df = pd.read_csv('enriched_australian_users_items.csv')

user_id = '76561197970982479'  
main_task_two(df, user_id)


协同过滤推荐： ['Mortal Kombat Komplete Edition', 'Cities in Motion 2', 'PixelJunk™ Monsters Ultimate', 'Tactical Intervention', 'Fate of the World']
